/phone/NORMAL_WALK/


Subject : ID_122
/HDD/DR_dataset/Alcogait1/ID_122
walk_6
 Walk : 6
     Device : phone
walk_1
 Walk : 1
     Device : phone
walk_11
 Walk : 11
     Device : phone
walk_14
 Walk : 14
     Device : phone
walk_19
 Walk : 19
     Device : phone
walk_17
 Walk : 17
     Device : phone
walk_10
 Walk : 10
     Device : phone
walk_4
 Walk : 4
     Device : phone
walk_8
 Walk : 8
     Device : phone
walk_5
 Walk : 5
     Device : phone
walk_15
 Walk : 15
     Device : phone
walk_18
 Walk : 18
     Device : phone
walk_13
 Walk : 13
     Device : phone
walk_3
 Walk : 3
     Device : phone
walk_12
 Walk : 12
     Device : phone
walk_20
 Walk : 20
     Device : phone
walk_16
 Walk : 16
     Device : phone
walk_9
 Walk : 9
     Device : phone
walk_7
 Walk : 7
     Device : phone
walk_2
 Walk : 2
     Device : phone
Subject : ID_159
/HDD/DR_dataset/Alcogait1/ID_159
walk_6
 Walk : 6
     Device : phone
walk_1
 Walk : 1
     Device : phone
walk_11
 Walk : 11
     Device : phone
walk_22
 Walk : 22
     D

KeyboardInterrupt: 

In [ ]:
import os
import numpy as np
import datetime
import time
import pandas
import pdb
from scipy.interpolate import interp1d
import pickle
import statistics
from scipy.signal import butter, filtfilt
from scipy.stats import zscore, kurtosis, skew
from utils import calculate_ellipse_area, create_ellipse, energy_peak, zero_crossing_rate, energy, sma, percentile
# find_velocity

dataDir = "/home/rli2/Alcogait3/"

if not os.path.isdir(saveDir):
    os.mkdir(saveDir)
# 
# phoneDir = "/phone/NORMAL_WALK/"
signaltype = 'NORMAL_WALK'
phoneDir = "/phone/"+signaltype+'/'
devices = ["phone"]
print(phoneDir)
semgementation_seconds=8
print('current working directory'+os.getcwd())
print('signal type is'+signaltype)
print('segmentation is '+str(semgementation_seconds)+'seconds')


compass = False
watch = False
sensors = ["accelerometer.csv","gyroscope.csv"]

if compass:
    sensors.append("compass.csv")

if watch:
    watchDir = "/watch/NORMAL_WALK/"
    devices.append("watch")
    # sensors.append("heart_rate.csv")  #Weirdly has far lesser values than watch acc and gyro readings

axes = ['x','y','z']
totalBAC = []
WholeData = []
samples = 128

samplingType = 'segments'
overlap_rate = 0.5

def noise_filter(data):
    Fs = 202.5
    N = 3
    cutoff = [5, 10, 15, 20]
    wn = [x*2/Fs for x in cutoff]
    b, a = butter(N, wn[0])
    data_filt = filtfilt(b, a, data)
    
    return data_filt


def segmentData(Data, Timestamp, walkId, subject, walkBAC, seg_time = semgementation_seconds , overlap_rate = overlap_rate , num_segments = None):
    Walk = []
    
    if Timestamp[-1] - Timestamp[0] <= 8:
        gyro_sway_a1 = []
        gyro_sway_b1 = []
        gyro_sway_c1 = []
        acc_sway_a1 = []
        acc_sway_b1 = []
        acc_sway_c1 = []

        Times = np.asarray(Timestamp)
        Times = np.remainder(Times,Times[0])
        Ttime = Times[-1]
        Values = np.asarray(Data,dtype = float)
        seg_data = np.empty((samples,0),float)


        acc_x_seg = Values[:,0]
        acc_y_seg = Values[:,1]
        acc_z_seg = Values[:,2]

        gyro_x_seg = Values[:,3]
        gyro_y_seg = Values[:,4]
        gyro_z_seg = Values[:,5]

        acc_rx_ct = max(acc_x_seg) - min(acc_x_seg)
        acc_ry_ct = max(acc_y_seg) - min(acc_y_seg)
        acc_rz_ct = max(acc_z_seg) - min(acc_z_seg)
        acc_stdx_ct   = statistics.stdev(acc_x_seg)
        acc_stdy_ct = statistics.stdev(acc_y_seg)
        acc_stdz_ct = statistics.stdev(acc_z_seg)
        acc_rmsx_ct = np.sqrt(np.mean(acc_x_seg ** 2))
        acc_rmsy_ct = np.sqrt(np.mean(acc_y_seg ** 2))
        acc_rmsz_ct = np.sqrt(np.mean(acc_z_seg ** 2))
        acc_kurtx_ct = kurtosis(acc_x_seg)
        acc_kurty_ct = kurtosis(acc_y_seg)
        acc_kurtz_ct = kurtosis(acc_z_seg)
        acc_energypeakX_ct, acc_freqspeakX_ct = energy_peak(acc_x_seg)
        acc_energypeakY_ct, acc_freqspeakY_ct = energy_peak(acc_y_seg)
        acc_energypeakZ_ct, acc_freqspeakZ_ct = energy_peak(acc_z_seg)                
        acc_skx_ct = skew(acc_x_seg)
        acc_sky_ct = skew(acc_y_seg)
        acc_skz_ct = skew(acc_z_seg)
        acc_a1 = create_ellipse(acc_x_seg, acc_y_seg)
        acc_b1 = create_ellipse(acc_y_seg, acc_z_seg)
        acc_c1 = create_ellipse(acc_z_seg, acc_x_seg)


        # Modified
        acc_sway_a1.append(acc_a1)
        acc_sway_b1.append(acc_b1)
        acc_sway_c1.append(acc_c1)



        gyro_rx_ct = max(gyro_x_seg) - min(gyro_x_seg)
        gyro_ry_ct = max(gyro_y_seg) - min(gyro_y_seg)
        gyro_rz_ct = max(gyro_z_seg) - min(gyro_z_seg)
        gyro_stdx_ct   = statistics.stdev(gyro_x_seg)
        gyro_stdy_ct = statistics.stdev(gyro_y_seg)
        gyro_stdz_ct = statistics.stdev(gyro_z_seg)
        gyro_rmsx_ct = np.sqrt(np.mean(gyro_x_seg ** 2))
        gyro_rmsy_ct = np.sqrt(np.mean(gyro_y_seg ** 2))
        gyro_rmsz_ct = np.sqrt(np.mean(gyro_z_seg ** 2))
        gyro_kurtx_ct = kurtosis(gyro_x_seg)
        gyro_kurty_ct = kurtosis(gyro_y_seg)
        gyro_kurtz_ct = kurtosis(gyro_z_seg)
        gyro_energypeakX_ct, gyro_freqspeakX_ct = energy_peak(gyro_x_seg)
        gyro_energypeakY_ct, gyro_freqspeakY_ct = energy_peak(gyro_y_seg)
        gyro_energypeakZ_ct, gyro_freqspeakZ_ct = energy_peak(gyro_z_seg)                
        gyro_skx_ct = skew(gyro_x_seg)
        gyro_sky_ct = skew(gyro_y_seg)
        gyro_skz_ct = skew(gyro_z_seg)
        gyro_a1 = create_ellipse(gyro_x_seg, gyro_y_seg)
        gyro_b1 = create_ellipse(gyro_y_seg, gyro_z_seg)
        gyro_c1 = create_ellipse(gyro_z_seg, gyro_x_seg)

        gyro_sway_a1.append(gyro_a1)
        gyro_sway_b1.append(gyro_b1)
        gyro_sway_c1.append(gyro_c1)

        # new features
        acc_a1_sway_per_second = float(acc_a1 /Ttime)
        acc_b1_sway_per_second = float(acc_b1 / Ttime)
        acc_c1_sway_per_second = float(acc_c1 / Ttime)

        gyro_a1_sway_per_second = float(gyro_a1 / Ttime)
        gyro_b1_sway_per_second = float(gyro_b1 / Ttime)
        gyro_c1_sway_per_second = float(gyro_c1 / Ttime)

        acc_x_zero_crossing_rate = zero_crossing_rate(acc_x_seg)
        acc_y_zero_crossing_rate = zero_crossing_rate(acc_y_seg)
        acc_z_zero_crossing_rate = zero_crossing_rate(acc_z_seg)

        gyro_x_zero_crossing_rate = zero_crossing_rate(gyro_x_seg)
        gyro_y_zero_crossing_rate = zero_crossing_rate(gyro_y_seg)
        gyro_z_zero_crossing_rate = zero_crossing_rate(gyro_z_seg)

        acc_x_energy = energy(acc_x_seg)
        acc_y_energy = energy(acc_y_seg)
        acc_z_energy = energy(acc_z_seg)

        gyro_x_energy = energy(gyro_x_seg)
        gyro_y_energy = energy(gyro_y_seg)
        gyro_z_energy = energy(gyro_z_seg)



        acc_signal_magnitude_area = sma(acc_x_seg,acc_y_seg,acc_z_seg)
        gyro_signal_magnitude_area = sma(gyro_x_seg, gyro_y_seg, gyro_z_seg)

        Walk.append([acc_rx_ct ,acc_ry_ct ,acc_rz_ct ,\
                    acc_stdx_ct ,acc_stdy_ct ,acc_stdz_ct ,\
                    acc_rmsx_ct ,acc_rmsy_ct ,acc_rmsz_ct ,\
                    acc_kurtx_ct,acc_kurty_ct,acc_kurtz_ct,\
                    acc_energypeakX_ct,acc_energypeakY_ct,acc_energypeakZ_ct,\
                    acc_freqspeakX_ct,acc_freqspeakY_ct,acc_freqspeakZ_ct,\
                    acc_skx_ct ,acc_sky_ct ,acc_skz_ct ,\
                    acc_a1 ,acc_b1 ,acc_c1,\
                    gyro_rx_ct ,gyro_ry_ct ,gyro_rz_ct ,\
                    gyro_stdx_ct ,gyro_stdy_ct ,gyro_stdz_ct ,\
                    gyro_rmsx_ct ,gyro_rmsy_ct ,gyro_rmsz_ct ,\
                    gyro_kurtx_ct,gyro_kurty_ct,gyro_kurtz_ct,\
                    gyro_energypeakX_ct,gyro_energypeakY_ct,gyro_energypeakZ_ct,\
                    gyro_freqspeakX_ct,gyro_freqspeakY_ct,gyro_freqspeakZ_ct,\
                    gyro_skx_ct ,gyro_sky_ct ,gyro_skz_ct ,\
                    gyro_a1 ,gyro_b1 ,gyro_c1,\
                    acc_a1_sway_per_second, acc_b1_sway_per_second, acc_c1_sway_per_second,\
                    gyro_a1_sway_per_second, gyro_b1_sway_per_second, gyro_c1_sway_per_second, \
                    acc_x_zero_crossing_rate, acc_y_zero_crossing_rate, acc_z_zero_crossing_rate, \
                    gyro_x_zero_crossing_rate, gyro_y_zero_crossing_rate, gyro_z_zero_crossing_rate,\
                    acc_x_energy ,acc_y_energy ,acc_z_energy ,\
                    gyro_x_energy, gyro_y_energy, gyro_z_energy, \
                    acc_signal_magnitude_area, gyro_signal_magnitude_area, \

                    'walk_'+str(walkId), subject, walkBAC])


        acc_xy_sway_dispersion = statistics.pvariance(acc_sway_a1)
        acc_yz_sway_dispersion = statistics.pvariance(acc_sway_b1)
        acc_xz_sway_dispersion = statistics.pvariance(acc_sway_c1)

        gyro_xy_sway_dispersion = statistics.pvariance(gyro_sway_a1)
        gyro_yz_sway_dispersion = statistics.pvariance(gyro_sway_b1)
        gyro_xz_sway_dispersion = statistics.pvariance(gyro_sway_c1)

        acc_sway_xy_percentile25, acc_sway_xy_percentile50, acc_sway_xy_percentile75 = percentile(acc_sway_a1)
        acc_sway_yz_percentile25, acc_sway_yz_percentile50, acc_sway_yz_percentile75 = percentile(acc_sway_b1)
        acc_sway_xz_percentile25, acc_sway_xz_percentile50, acc_sway_xz_percentile75 = percentile(acc_sway_c1)

        gyro_sway_xy_percentile25, gyro_sway_xy_percentile50, gyro_sway_xy_percentile75 = percentile(gyro_sway_a1)
        gyro_sway_yz_percentile25, gyro_sway_yz_percentile50, gyro_sway_yz_percentile75 = percentile(gyro_sway_b1)
        gyro_sway_xz_percentile25, gyro_sway_xz_percentile50, gyro_sway_xz_percentile75 = percentile(gyro_sway_c1)
        walk_seg_num = np.size(Walk,0)
        # Walk = np.asarray(Walk)

        Walk =np.column_stack(
            (Walk,list(np.repeat(acc_xy_sway_dispersion,walk_seg_num)),list(np.repeat(acc_yz_sway_dispersion,walk_seg_num)),
             list(np.repeat(acc_xz_sway_dispersion, walk_seg_num)),list(np.repeat(gyro_xy_sway_dispersion,walk_seg_num)),
             list(np.repeat(gyro_yz_sway_dispersion, walk_seg_num)),list(np.repeat(gyro_xz_sway_dispersion,walk_seg_num)),
             list(np.repeat(acc_sway_xy_percentile25, walk_seg_num)),list(np.repeat(acc_sway_xy_percentile50,walk_seg_num)),list(np.repeat(acc_sway_xy_percentile75, walk_seg_num)),
             list(np.repeat(acc_sway_yz_percentile25,walk_seg_num)),list(np.repeat(acc_sway_yz_percentile50, walk_seg_num)),list(np.repeat(acc_sway_yz_percentile75,walk_seg_num)),
             list(np.repeat(acc_sway_xz_percentile25, walk_seg_num)),list(np.repeat(acc_sway_xz_percentile50, walk_seg_num)),list(np.repeat(acc_sway_xz_percentile75, walk_seg_num)),
             list(np.repeat(gyro_sway_xz_percentile25,walk_seg_num)),list(np.repeat(gyro_sway_xy_percentile50, walk_seg_num)),list(np.repeat(gyro_sway_xy_percentile75,walk_seg_num)),
             list(np.repeat(gyro_sway_yz_percentile25, walk_seg_num)),list(np.repeat(gyro_sway_yz_percentile50,walk_seg_num)),list(np.repeat(gyro_sway_yz_percentile75, walk_seg_num)),
             list(np.repeat(gyro_sway_xz_percentile25,walk_seg_num)),list(np.repeat(gyro_sway_xz_percentile50, walk_seg_num)),list(np.repeat(gyro_sway_xz_percentile75, walk_seg_num)),

             ))

#         return Walk

    
    else:

        Times = []
        Values = []
        start = Timestamp[0]
        i = 0
        start_i = i
        gyro_sway_a1 = []
        gyro_sway_b1 = []
        gyro_sway_c1 = []
        acc_sway_a1 = []
        acc_sway_b1 = []
        acc_sway_c1 = []
        
              
        while i < len(Timestamp):
            Values.append(Data[i])
            Times.append(Timestamp[i])

            if Timestamp[i] - start >= 8:
#                 Ttime = Timestamp[i]
#                 print(Ttime)
                Times = np.asarray(Times)
                Times = np.remainder(Times,Times[0])
                Ttime = Times[-1]
                Values = np.asarray(Values,dtype = float)
                seg_data = np.empty((samples,0),float)


                acc_x_seg = Values[:,0]
                acc_y_seg = Values[:,1]
                acc_z_seg = Values[:,2]

                gyro_x_seg = Values[:,3]
                gyro_y_seg = Values[:,4]
                gyro_z_seg = Values[:,5]

                acc_rx_ct = max(acc_x_seg) - min(acc_x_seg)
                acc_ry_ct = max(acc_y_seg) - min(acc_y_seg)
                acc_rz_ct = max(acc_z_seg) - min(acc_z_seg)
                acc_stdx_ct   = statistics.stdev(acc_x_seg)
                acc_stdy_ct = statistics.stdev(acc_y_seg)
                acc_stdz_ct = statistics.stdev(acc_z_seg)
                acc_rmsx_ct = np.sqrt(np.mean(acc_x_seg ** 2))
                acc_rmsy_ct = np.sqrt(np.mean(acc_y_seg ** 2))
                acc_rmsz_ct = np.sqrt(np.mean(acc_z_seg ** 2))
                acc_kurtx_ct = kurtosis(acc_x_seg)
                acc_kurty_ct = kurtosis(acc_y_seg)
                acc_kurtz_ct = kurtosis(acc_z_seg)
                acc_energypeakX_ct, acc_freqspeakX_ct = energy_peak(acc_x_seg)
                acc_energypeakY_ct, acc_freqspeakY_ct = energy_peak(acc_y_seg)
                acc_energypeakZ_ct, acc_freqspeakZ_ct = energy_peak(acc_z_seg)                
                acc_skx_ct = skew(acc_x_seg)
                acc_sky_ct = skew(acc_y_seg)
                acc_skz_ct = skew(acc_z_seg)
                acc_a1 = create_ellipse(acc_x_seg, acc_y_seg)
                acc_b1 = create_ellipse(acc_y_seg, acc_z_seg)
                acc_c1 = create_ellipse(acc_z_seg, acc_x_seg)


                # Modified
                acc_sway_a1.append(acc_a1)
                acc_sway_b1.append(acc_b1)
                acc_sway_c1.append(acc_c1)



                gyro_rx_ct = max(gyro_x_seg) - min(gyro_x_seg)
                gyro_ry_ct = max(gyro_y_seg) - min(gyro_y_seg)
                gyro_rz_ct = max(gyro_z_seg) - min(gyro_z_seg)
                gyro_stdx_ct   = statistics.stdev(gyro_x_seg)
                gyro_stdy_ct = statistics.stdev(gyro_y_seg)
                gyro_stdz_ct = statistics.stdev(gyro_z_seg)
                gyro_rmsx_ct = np.sqrt(np.mean(gyro_x_seg ** 2))
                gyro_rmsy_ct = np.sqrt(np.mean(gyro_y_seg ** 2))
                gyro_rmsz_ct = np.sqrt(np.mean(gyro_z_seg ** 2))
                gyro_kurtx_ct = kurtosis(gyro_x_seg)
                gyro_kurty_ct = kurtosis(gyro_y_seg)
                gyro_kurtz_ct = kurtosis(gyro_z_seg)
                gyro_energypeakX_ct, gyro_freqspeakX_ct = energy_peak(gyro_x_seg)
                gyro_energypeakY_ct, gyro_freqspeakY_ct = energy_peak(gyro_y_seg)
                gyro_energypeakZ_ct, gyro_freqspeakZ_ct = energy_peak(gyro_z_seg)                
                gyro_skx_ct = skew(gyro_x_seg)
                gyro_sky_ct = skew(gyro_y_seg)
                gyro_skz_ct = skew(gyro_z_seg)
                gyro_a1 = create_ellipse(gyro_x_seg, gyro_y_seg)
                gyro_b1 = create_ellipse(gyro_y_seg, gyro_z_seg)
                gyro_c1 = create_ellipse(gyro_z_seg, gyro_x_seg)

                gyro_sway_a1.append(gyro_a1)
                gyro_sway_b1.append(gyro_b1)
                gyro_sway_c1.append(gyro_c1)
                
                # new features
                acc_a1_sway_per_second = float(acc_a1 /Ttime)
                acc_b1_sway_per_second = float(acc_b1 / Ttime)
                acc_c1_sway_per_second = float(acc_c1 / Ttime)

                gyro_a1_sway_per_second = float(gyro_a1 / Ttime)
                gyro_b1_sway_per_second = float(gyro_b1 / Ttime)
                gyro_c1_sway_per_second = float(gyro_c1 / Ttime)

                acc_x_zero_crossing_rate = zero_crossing_rate(acc_x_seg)
                acc_y_zero_crossing_rate = zero_crossing_rate(acc_y_seg)
                acc_z_zero_crossing_rate = zero_crossing_rate(acc_z_seg)

                gyro_x_zero_crossing_rate = zero_crossing_rate(gyro_x_seg)
                gyro_y_zero_crossing_rate = zero_crossing_rate(gyro_y_seg)
                gyro_z_zero_crossing_rate = zero_crossing_rate(gyro_z_seg)

                acc_x_energy = energy(acc_x_seg)
                acc_y_energy = energy(acc_y_seg)
                acc_z_energy = energy(acc_z_seg)

                gyro_x_energy = energy(gyro_x_seg)
                gyro_y_energy = energy(gyro_y_seg)
                gyro_z_energy = energy(gyro_z_seg)



                acc_signal_magnitude_area = sma(acc_x_seg,acc_y_seg,acc_z_seg)
                gyro_signal_magnitude_area = sma(gyro_x_seg, gyro_y_seg, gyro_z_seg)
                
                ## ADD NEW FEATURES HERE
                # print(acc_a1)
                # acc_xy_sway_dispersion = statistics.pvariance(acc_a1)
                # acc_yz_sway_dispersion = statistics.pvariance(acc_b1)
                # acc_xz_sway_dispersion = statistics.pvariance(acc_c1)
                #
                # gyro_xy_sway_dispersion = statistics.pvariance(gyro_a1)
                # gyro_yz_sway_dispersion = statistics.pvariance(gyro_b1)
                # gyro_xz_sway_dispersion = statistics.pvariance(gyro_c1)
                #
                # acc_sway_xy_percentile25, acc_sway_xy_percentile50, acc_sway_xy_percentile75 = percentile(acc_a1)
                # acc_sway_yz_percentile25, acc_sway_yz_percentile50, acc_sway_yz_percentile75 = percentile(acc_b1)
                # acc_sway_xz_percentile25, acc_sway_xz_percentile50, acc_sway_xz_percentile75 = percentile(acc_c1)
                #
                # gyro_sway_xy_percentile25, gyro_sway_xy_percentile50, gyro_sway_xy_percentile75 = percentile(gyro_a1)
                # gyro_sway_yz_percentile25, gyro_sway_yz_percentile50, gyro_sway_yz_percentile75 = percentile(gyro_b1)
                # gyro_sway_xz_percentile25, gyro_sway_xz_percentile50, gyro_sway_xz_percentile75 = percentile(gyro_c1)

                Walk.append([acc_rx_ct ,acc_ry_ct ,acc_rz_ct ,\
                            acc_stdx_ct ,acc_stdy_ct ,acc_stdz_ct ,\
                            acc_rmsx_ct ,acc_rmsy_ct ,acc_rmsz_ct ,\
                            acc_kurtx_ct,acc_kurty_ct,acc_kurtz_ct,\
                            acc_energypeakX_ct,acc_energypeakY_ct,acc_energypeakZ_ct,\
                            acc_freqspeakX_ct,acc_freqspeakY_ct,acc_freqspeakZ_ct,\
                            acc_skx_ct ,acc_sky_ct ,acc_skz_ct ,\
                            acc_a1 ,acc_b1 ,acc_c1,\
                            gyro_rx_ct ,gyro_ry_ct ,gyro_rz_ct ,\
                            gyro_stdx_ct ,gyro_stdy_ct ,gyro_stdz_ct ,\
                            gyro_rmsx_ct ,gyro_rmsy_ct ,gyro_rmsz_ct ,\
                            gyro_kurtx_ct,gyro_kurty_ct,gyro_kurtz_ct,\
                            gyro_energypeakX_ct,gyro_energypeakY_ct,gyro_energypeakZ_ct,\
                            gyro_freqspeakX_ct,gyro_freqspeakY_ct,gyro_freqspeakZ_ct,\
                            gyro_skx_ct ,gyro_sky_ct ,gyro_skz_ct ,\
                            gyro_a1 ,gyro_b1 ,gyro_c1,\
                            acc_a1_sway_per_second, acc_b1_sway_per_second, acc_c1_sway_per_second,\
                            gyro_a1_sway_per_second, gyro_b1_sway_per_second, gyro_c1_sway_per_second, \
                            acc_x_zero_crossing_rate, acc_y_zero_crossing_rate, acc_z_zero_crossing_rate, \
                            gyro_x_zero_crossing_rate, gyro_y_zero_crossing_rate, gyro_z_zero_crossing_rate,\
                            acc_x_energy ,acc_y_energy ,acc_z_energy ,\
                            gyro_x_energy, gyro_y_energy, gyro_z_energy, \
                            acc_signal_magnitude_area, gyro_signal_magnitude_area, \

                            'walk_'+str(walkId), subject, walkBAC])


                i = start_i + np.argmax(Times>4)
                start_i = i 
                start = Timestamp[i]
                Values = []
                Times = []
                
                continue


            i += 1

        acc_xy_sway_dispersion = statistics.pvariance(acc_sway_a1)
        acc_yz_sway_dispersion = statistics.pvariance(acc_sway_b1)
        acc_xz_sway_dispersion = statistics.pvariance(acc_sway_c1)

        gyro_xy_sway_dispersion = statistics.pvariance(gyro_sway_a1)
        gyro_yz_sway_dispersion = statistics.pvariance(gyro_sway_b1)
        gyro_xz_sway_dispersion = statistics.pvariance(gyro_sway_c1)

        acc_sway_xy_percentile25, acc_sway_xy_percentile50, acc_sway_xy_percentile75 = percentile(acc_sway_a1)
        acc_sway_yz_percentile25, acc_sway_yz_percentile50, acc_sway_yz_percentile75 = percentile(acc_sway_b1)
        acc_sway_xz_percentile25, acc_sway_xz_percentile50, acc_sway_xz_percentile75 = percentile(acc_sway_c1)

        gyro_sway_xy_percentile25, gyro_sway_xy_percentile50, gyro_sway_xy_percentile75 = percentile(gyro_sway_a1)
        gyro_sway_yz_percentile25, gyro_sway_yz_percentile50, gyro_sway_yz_percentile75 = percentile(gyro_sway_b1)
        gyro_sway_xz_percentile25, gyro_sway_xz_percentile50, gyro_sway_xz_percentile75 = percentile(gyro_sway_c1)
        walk_seg_num = np.size(Walk,0)
        
        Walk =np.column_stack(
            (Walk,list(np.repeat(acc_xy_sway_dispersion,walk_seg_num)),list(np.repeat(acc_yz_sway_dispersion,walk_seg_num)),
             list(np.repeat(acc_xz_sway_dispersion, walk_seg_num)),list(np.repeat(gyro_xy_sway_dispersion,walk_seg_num)),
             list(np.repeat(gyro_yz_sway_dispersion, walk_seg_num)),list(np.repeat(gyro_xz_sway_dispersion,walk_seg_num)),
             list(np.repeat(acc_sway_xy_percentile25, walk_seg_num)),list(np.repeat(acc_sway_xy_percentile50,walk_seg_num)),list(np.repeat(acc_sway_xy_percentile75, walk_seg_num)),
             list(np.repeat(acc_sway_yz_percentile25,walk_seg_num)),list(np.repeat(acc_sway_yz_percentile50, walk_seg_num)),list(np.repeat(acc_sway_yz_percentile75,walk_seg_num)),
             list(np.repeat(acc_sway_xz_percentile25, walk_seg_num)),list(np.repeat(acc_sway_xz_percentile50, walk_seg_num)),list(np.repeat(acc_sway_xz_percentile75, walk_seg_num)),
             list(np.repeat(gyro_sway_xz_percentile25,walk_seg_num)),list(np.repeat(gyro_sway_xy_percentile50, walk_seg_num)),list(np.repeat(gyro_sway_xy_percentile75,walk_seg_num)),
             list(np.repeat(gyro_sway_yz_percentile25, walk_seg_num)),list(np.repeat(gyro_sway_yz_percentile50,walk_seg_num)),list(np.repeat(gyro_sway_yz_percentile75, walk_seg_num)),
             list(np.repeat(gyro_sway_xz_percentile25,walk_seg_num)),list(np.repeat(gyro_sway_xz_percentile50, walk_seg_num)),list(np.repeat(gyro_sway_xz_percentile75, walk_seg_num)),

             ))

    return Walk

for subject in next(os.walk(dataDir))[1]:
    
    subjectBAC = []
    print("Subject : " + subject)
    WholeWalk = []
    print(dataDir+subject)
    for walk in next(os.walk(dataDir+subject))[1]:
        print(walk)
        walkId = walk.split('_')[1]
        print(" Walk : " + walkId)
        for device in devices:
            print("     Device : " + device)
            Time = None
            Data = None
            headers = []

            if device == "phone":
                deviceDir = phoneDir
            
            if device == "watch":
                deviceDir = watchDir

            for sensor in sensors:
                # print(dataDir,subject,walk,deviceDir,sensor)
                file = dataDir+subject+"/"+walk+deviceDir+sensor        
                
                if os.path.exists(file):
                    walkBAC = float(pandas.read_csv(file, nrows=0).columns[1])  
                    sensorData = pandas.read_csv(file, skiprows = 2)
                    sensorData=sensorData.dropna()
                    Timestamp = sensorData.values[:,-1]
                    
                    if sensor == "heart_rate.csv":
                        Values = np.asarray(sensorData.values[:,1:2])
                        headers = headers + [sensor.split(".")[0]]

                    else:
                        Values = np.asarray(sensorData.values[:,1:4])
                        headers = headers + [sensor.split(".")[0] + axis for axis in axes]

                    del sensorData

                    ## Use if You want to use butterworth filter

                    if Values.shape[0]:
                        Values[:,0] = noise_filter(Values[:,0])
                        Values[:,1] = noise_filter(Values[:,1])
                        Values[:,2] = noise_filter(Values[:,2])   

                        Timestamp = [times.split(' ')[1] for times in Timestamp]
                        Timestamp = [[time.strptime(times.split('.')[0],'%H:%M:%S'),\
                                        float(times.split('.')[1])/1000.0] for times in Timestamp]
                        Timestamp = np.asarray([datetime.timedelta(hours=times[0].tm_hour,minutes=times[0].tm_min,\
                                        seconds=times[0].tm_sec).total_seconds()+times[1] for times in Timestamp])[:,np.newaxis]

                        if hasattr(Time, 'shape'):
                            Time = np.append(Time, Timestamp, axis = 1)
                            Data = np.append(Data, Values, axis = 1)

                        else:
                            Time = Timestamp.copy()
                            Data = Values.copy()                    

                        del Timestamp,Values

                else:
                    break
                    
        if not hasattr(Data, 'shape'):
            continue
            
        num_sensors = Data.shape[1]
        
        if num_sensors < 6: 
            continue

        Timestamp = np.mean(Time, axis = 1)
        
        if samplingType == 'Walk':
            Times = np.remainder(Timestamp,Timestamp[0])        
            seg_data = np.empty((samples,0),float)
                    
            for i in range(num_sensors):
            
                sensor_data = Data[:,i]
                sensorInterp = interp1d(Times, sensor_data)
                sensorInterpTime = np.linspace(0.0, Times[-1], samples)
                sensorInterpVal = sensorInterp(sensorInterpTime)
                seg_data = np.append(seg_data,np.asarray(sensorInterpVal,dtype = float)[:,np.newaxis],axis = 1)                     
            
            Walk = [seg_data.copy()]
        
        if samplingType == 'segments':
            Walk = segmentData(Data, Timestamp, walkId, subject, walkBAC)

            # Walk = segmentData(Data, Timestamp, walkId, subject, walkBAC, num_segments = 32)
        # pdb.set_trace()
        
        WholeWalk.append(Walk)
        # subjectBAC.append(walkBAC)
    
    # totalBAC.append(subjectBAC)
    WholeData.append(WholeWalk)

os.chdir(saveDir)
print('saving into'+os.getcwd())
with open('Data_features_seg'+str(semgementation_seconds)+signaltype+'up.pkl', 'wb') as f:
    pickle.dump(WholeData, f)

with open('Labels_features_seg'+str(semgementation_seconds)+signaltype+'up.pkl', 'wb') as f:
    pickle.dump(totalBAC, f)

/phone/NORMAL_WALK/
current working directory/home/gbalakrishnan/Complete Dataset/New paper test/result_features_Alco
signal type isNORMAL_WALK
segmentation is 8seconds
Subject : ID_129
/home/gbalakrishnan/Complete Dataset/Up/ID_129
walk_7
 Walk : 7
     Device : phone
walk_6
 Walk : 6
     Device : phone
walk_5
 Walk : 5
     Device : phone
walk_4
 Walk : 4
     Device : phone


/phone/HEEL_TO_TOE/
current working directory/home/gbalakrishnan/Complete Dataset/New paper test
signal type isHEEL_TO_TOE
segmentation is 8seconds
Subject : ID_165
Down_HT/ID_165
walk_22
 Walk : 22
     Device : phone
[]
hi
     Device : watch


In [1]:
import os
import numpy as np
import datetime
import time
import pandas as pd
import pdb
from scipy.interpolate import interp1d
import pickle
import statistics
from scipy.signal import butter, filtfilt
from scipy.stats import zscore, kurtosis, skew
from utils import calculate_ellipse_area, create_ellipse, energy_peak, zero_crossing_rate, energy, sma, percentile
# find_velocity
signaltype = 'STANDING_ON_ONE_FOOT'
phoneDir = "/phone/"+signaltype+'/'
devices = ["phone"]
print(phoneDir)
semgementation_seconds=8
df_data_path = '/home/gbalakrishnan/Complete Dataset/New paper test/result_features_Alco/Entire/'+'Data_features_seg'+str(semgementation_seconds)+signaltype
df_label_path = '/home/gbalakrishnan/Complete Dataset/New paper test/result_features_Alco/Entire/'+'Labels_features_seg'+str(semgementation_seconds)+signaltype


/phone/STANDING_ON_ONE_FOOT/


In [7]:
type(df)

NameError: name 'df' is not defined

In [4]:
with open(df_data_path+'_down.pkl', "rb") as f:
    a = pickle.load(f)

with open(df_label_path+'_down.pkl', "rb") as f:
    b = pickle.load(f)

feature_name = ["acc_rx_ct", "acc_ry_ct", "acc_rz_ct",
                "acc_stdx_ct", "acc_stdy_ct", "acc_stdz_ct",
                "acc_rmsx_ct", "acc_rmsy_ct", "acc_rmsz_ct",
                "acc_kurtx_ct", "acc_kurty_ct", "acc_kurtz_ct",
                "acc_energypeakX_ct", "acc_energypeakY_ct", "acc_energypeakZ_ct",
                "acc_freqspeakX_ct", "acc_freqspeakY_ct", "acc_freqspeakZ_ct",
                "acc_skx_ct", "acc_sky_ct", "acc_skz_ct",
                "acc_eclipse_a1", "acc_eclipse_b1", "acc_eclipse_c1",
                "gyro_rx_ct", "gyro_ry_ct", "gyro_rz_ct",
                "gyro_stdx_ct", "gyro_stdy_ct", "gyro_stdz_ct",
                "gyro_rmsx_ct", "gyro_rmsy_ct", "gyro_rmsz_ct",
                "gyro_kurtx_ct", "gyro_kurty_ct","gyro_kurtz_ct",
                "gyro_energypeakX_ct", "gyro_energypeakY_ct", "gyro_energypeakZ_ct",
                "gyro_freqspeakX_ct", "gyro_freqspeakY_ct", "gyro_freqspeakZ_ct",
                "gyro_skx_ct", "gyro_sky_ct", "gyro_skz_ct",
                "gyro_eclipse_a1", "gyro_eclipse_b1", "gyro_eclipse_c1",
                "acc_a1_sway_per_second", "acc_b1_sway_per_second", "acc_c1_sway_per_second",
                "gyro_a1_sway_per_second", "gyro_b1_sway_per_second", "gyro_c1_sway_per_second",
                "acc_x_zero_crossing_rate", "acc_y_zero_crossing_rate", "acc_z_zero_crossing_rate",
                "gyro_x_zero_crossing_rate", "gyro_y_zero_crossing_rate", "gyro_z_zero_crossing_rate",
                "acc_x_energy", "acc_y_energy", "acc_z_energy",
                "gyro_x_energy", "gyro_y_energy", "gyro_z_energy",

                "acc_signal_magnitude_area", "gyro_signal_magnitude_area", "walk_ID", "subject_ID", "BAC_VALUE",
                "acc_xy_sway_dispersion", "acc_yz_sway_dispersion", "acc_xz_sway_dispersion",
                "gyro_xy_sway_dispersion", "gyro_yz_sway_dispersion", "gyro_xz_sway_dispersion",
                "acc_sway_xy_percentile25", "acc_sway_xy_percentile50", "acc_sway_xy_percentile75",
                "acc_sway_yz_percentile25", "acc_sway_yz_percentile50", "acc_sway_yz_percentile75",
                "acc_sway_xz_percentile25", "acc_sway_xz_percentile50", "acc_sway_xz_percentile75",
                "gyro_sway_xy_percentile25", "gyro_sway_xy_percentile50", "gyro_sway_xy_percentile75",
                "gyro_sway_yz_percentile25", "gyro_sway_yz_percentile50", "gyro_sway_yz_percentile75",
                "gyro_sway_xz_percentile25", "gyro_sway_xz_percentile50", "gyro_sway_xz_percentile75",
                ]

# feature_name = ["acc_rx_ct", "acc_ry_ct", "acc_rz_ct",
#                 "acc_stdx_ct", "acc_stdy_ct", "acc_stdz_ct",
#                 "acc_rmsx_ct", "acc_rmsy_ct", "acc_rmsz_ct",
#                 "acc_kurtx_ct", "acc_kurty_ct", "acc_kurtz_ct",
#                 "acc_energypeakX_ct", "acc_energypeakY_ct", "acc_energypeakZ_ct",
#                 "acc_freqspeakX_ct", "acc_freqspeakY_ct", "acc_freqspeakZ_ct",
#                 "acc_skx_ct", "acc_sky_ct", "acc_skz_ct",
#                 "acc_eclipse_a1", "acc_eclipse_b1", "acc_eclipse_c1",
#                 "gyro_rx_ct", "gyro_ry_ct", "gyro_rz_ct",
#                 "gyro_stdx_ct", "gyro_stdy_ct", "gyro_stdz_ct",
#                 "gyro_rmsx_ct", "gyro_rmsy_ct", "gyro_rmsz_ct",
#                 "gyro_kurtx_ct", "gyro_kurty_ct","gyro_kurtz_ct",
#                 "gyro_energypeakX_ct", "gyro_energypeakY_ct", "gyro_energypeakZ_ct",
#                 "gyro_freqspeakX_ct", "gyro_freqspeakY_ct", "gyro_freqspeakZ_ct",
#                 "gyro_skx_ct", "gyro_sky_ct", "gyro_skz_ct",
#                 "gyro_eclipse_a1", "gyro_eclipse_b1", "gyro_eclipse_c1",
#                 "acc_a1_sway_per_second", "acc_b1_sway_per_second", "acc_c1_sway_per_second",
#                 "gyro_a1_sway_per_second", "gyro_b1_sway_per_second", "gyro_c1_sway_per_second",
#                 "acc_x_zero_crossing_rate", "acc_y_zero_crossing_rate", "acc_z_zero_crossing_rate",
#                 "gyro_x_zero_crossing_rate", "gyro_y_zero_crossing_rate", "gyro_z_zero_crossing_rate",
#                 "acc_x_energy", "acc_y_energy", "acc_z_energy",
#                 "gyro_x_energy", "gyro_y_energy", "gyro_z_energy",

#                 "acc_signal_magnitude_area", "gyro_signal_magnitude_area", "walk_ID", "subject_ID", "BAC_VALUE",
#                 "acc_xy_sway_dispersion", "acc_yz_sway_dispersion", "acc_xz_sway_dispersion",
#                 "gyro_xy_sway_dispersion", "gyro_yz_sway_dispersion", "gyro_xz_sway_dispersion",
#                 "acc_sway_xy_percentile25", "acc_sway_xy_percentile50", "acc_sway_xy_percentile75",
#                 "acc_sway_yz_percentile25", "acc_sway_yz_percentile50", "acc_sway_yz_percentile75",
#                 "acc_sway_xz_percentile25", "acc_sway_xz_percentile50", "acc_sway_xz_percentile75",
#                 "gyro_sway_xy_percentile25", "gyro_sway_xy_percentile50", "gyro_sway_xy_percentile75",
#                 "gyro_sway_yz_percentile25", "gyro_sway_yz_percentile50", "gyro_sway_yz_percentile75",
#                 "gyro_sway_xz_percentile25", "gyro_sway_xz_percentile50", "gyro_sway_xz_percentile75",
#                 ]

Data = []
for subject_data in a:
    for walk_data in subject_data:
        for segment_data in walk_data:
            Data.append(segment_data)
# pdb.set_trace()
print(len(Data[0]))
df = pd.DataFrame(data=Data, columns=feature_name)
print(df.shape)

95
(2124, 95)


In [5]:
df[df.subject_ID == 'ID_145'].walk_ID

24     walk_9
25     walk_9
26     walk_9
27     walk_9
28     walk_9
29     walk_9
30     walk_9
31     walk_9
32     walk_9
33     walk_8
34     walk_8
35     walk_8
36     walk_8
37     walk_8
38     walk_8
39     walk_8
40     walk_8
41     walk_8
42    walk_15
43    walk_15
44    walk_15
45    walk_15
46    walk_15
47    walk_15
48    walk_15
49    walk_15
50    walk_12
51    walk_12
52    walk_12
53    walk_12
54    walk_12
55    walk_12
56    walk_12
57    walk_12
58    walk_12
Name: walk_ID, dtype: object

In [135]:
corrupt_IDs = ['ID_126', 'ID_138', 'ID_144', 'ID_162', 'ID_177']

for ID in corrupt_IDs:
    df = df[df.subject_ID != ID]
print(df.shape)

(2304, 95)


In [136]:
import statsmodels.api as sm
def featureSelect(X,  savePath = None, initial_list=[], threshold_out=0.05, verbose=True):
    # "walk_ID", "subject_ID"
    y = X['BAC_VALUE'].values
    X = X.drop(columns=['subject_ID',  'walk_ID', 'BAC_Value'],
                    inplace=False)   

    excluded = list(initial_list)
    included = list(set(X.columns) - set(excluded))
      
       while True:    
           changed = False            
           model = sm.OLS(y.astype(float), sm.add_constant(pd.DataFrame(X[included])).astype(float)).fit()
           # use all coefs except intercept
           pvalues = model.pvalues.iloc[1:]
           worst_pval = pvalues.max()  # null if pvalues is empty
           
           if worst_pval > threshold_out:
               changed = True
               worst_feature = pvalues.argmax()
               included.remove(worst_feature)
           
               if verbose:
                   print('Drop {:30} with p-value {:.6}'.format(worst_feature, worst_pval))
           
           if not changed:
               break

       x = X.loc[:, included].values
  
       # if savePath:
       #  saveFeatures(x, y, savePath)

       return x, y

In [137]:
df

,acc_rx_ct,acc_ry_ct,acc_rz_ct,acc_stdx_ct,acc_stdy_ct,acc_stdz_ct,acc_rmsx_ct,acc_rmsy_ct,acc_rmsz_ct,acc_kurtx_ct,...,acc_sway_xz_percentile75,gyro_sway_xy_percentile25,gyro_sway_xy_percentile50,gyro_sway_xy_percentile75,gyro_sway_yz_percentile25,gyro_sway_yz_percentile50,gyro_sway_yz_percentile75,gyro_sway_xz_percentile25,gyro_sway_xz_percentile50,gyro_sway_xz_percentile75
0,2.16987439845,1.65452908733,1.47156579126,0.374431246744,0.20568583424,0.295769200133,1.27001075183,9.23841491253,2.69847075618,4.53761106242,...,7.65474549327,0.00825967554901,0.0150481594105,0.0169481586861,0.0114222177985,0.0133019623512,0.0170483769145,0.00825967554901,0.00858601810416,0.0110721859852
1,0.692519713487,0.539827971159,0.586485675309,0.099923223005,0.0821162943489,0.0986339950804,1.20174907709,9.30590186256,2.47151176842,0.697862091844,...,7.65474549327,0.00825967554901,0.0150481594105,0.0169481586861,0.0114222177985,0.0133019623512,0.0170483769145,0.00825967554901,0.00858601810416,0.0110721859852
2,0.623237432472,0.814812477454,0.482113467189,0.102187096742,0.110076150434,0.09168680926,1.22152809496,9.33124436995,2.35766756727,0.75647151463,...,7.65474549327,0.00825967554901,0.0150481594105,0.0169481586861,0.0114222177985,0.0133019623512,0.0170483769145,0.00825967554901,0.00858601810416,0.0110721859852
3,0.721188469439,0.814812477454,0.518790037259,0.114576409358,0.109080366574,0.105376671438,1.2867014255,9.31849353475,2.37392085201,1.30800231506,...,7.65474549327,0.00825967554901,0.0150481594105,0.0169481586861,0.0114222177985,0.0133019623512,0.0170483769145,0.00825967554901,0.00858601810416,0.0110721859852
4,0.619086920011,0.447202038765,0.518790037259,0.104539142654,0.0796519121893,0.0835222965636,1.38866458121,9.29364611402,2.42627814797,0.0321052027986,...,7.65474549327,0.00825967554901,0.0150481594105,0.0169481586861,0.0114222177985,0.0133019623512,0.0170483769145,0.00825967554901,0.00858601810416,0.0110721859852
5,0.774065344202,0.568075803902,0.565652324256,0.116755479801,0.0938005050547,0.100178553275,1.39895556634,9.30529437537,2.37558649245,1.07165541995,...,7.65474549327,0.00825967554901,0.0150481594105,0.0169481586861,0.0114222177985,0.0133019623512,0.0170483769145,0.00825967554901,0.00858601810416,0.0110721859852
6,0.820598554631,0.579551545887,0.565652324256,0.113540382323,0.0999731011169,0.0923011229624,1.41325535691,9.31786588417,2.3160998162,1.81805335013,...,7.65474549327,0.00825967554901,0.0150481594105,0.0169481586861,0.0114222177985,0.0133019623512,0.0170483769145,0.00825967554901,0.00858601810416,0.0110721859852
7,0.645489843629,0.625036882088,0.513238499739,0.100441019546,0.0991858151743,0.106714542291,1.4378662456,9.3301128165,2.24629008338,0.717205035895,...,7.65474549327,0.00825967554901,0.0150481594105,0.0169481586861,0.0114222177985,0.0133019623512,0.0170483769145,0.00825967554901,0.00858601810416,0.0110721859852
8,2.25966994284,1.69437143352,1.69855899158,0.41819510572,0.248231716015,0.40761702523,0.842112823083,9.17443918602,3.03976090899,4.08337900811,...,6.81821568959,0.0061579967801,0.0106824974118,0.0130907439721,0.0100799533061,0.0105742766817,0.0110511365545,0.0061579967801,0.00683201274148,0.00817423533111
9,0.545977144854,0.773955275173,0.463833598622,0.108000999188,0.0926574302896,0.086134713468,0.729044634401,9.25262797492,2.80812697549,-0.631740875784,...,6.81821568959,0.0061579967801,0.0106824974118,0.0130907439721,0.0100799533061,0.0105742766817,0.0110511365545,0.0061579967801,0.00683201274148,0.00817423533111


In [138]:
res =[]
from sklearn import preprocessing
from sklearn.feature_selection import SelectKBest, chi2, RFE
import math
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.metrics import mean_squared_error, accuracy_score
import xgboost as xgb
for i in range(10):
    split_rate=0.8

    allIDs = np.unique(df['subject_ID'].values)
    num_subjects = len(allIDs)
#     y = df[['subject_ID','walk_ID','BAC_VALUE']]
    y = df[['BAC_VALUE']].values
    X = df.drop(columns=['subject_ID', 'walk_ID','BAC_VALUE' ],
               inplace=False)


    id_string_selected = np.random.choice(allIDs, num_subjects, replace=False)
    train_number = int(round(num_subjects*split_rate))
    train_ID = np.random.choice(id_string_selected, train_number, replace=False)
    test_ID = np.setdiff1d(id_string_selected,train_ID)
    train_ID_index= df['subject_ID'].isin(train_ID)
    test_ID_index = df['subject_ID'].isin(test_ID)
    X,y = featureSelect(df)
    print(np.shape(X))
    X=preprocessing.normalize(X)
#     print(np.shape(X))
#     pca = PCA(2)
#     X = pca.fit_transform(X,y.values)
#     X = SelectKBest( k=20).fit_transform(X, y)
#     rg = xgb.XGBRegressor()
#     rg=RandomForestRegressor()
#     rfe = RFE(rg,20)
#     X = rfe.fit_transform(X, y)
    train_X=X[train_ID_index]
#     train_Y = y.loc[train_ID_index].drop(columns=['subject_ID', 'walk_ID'],
#                inplace=False).values
    train_Y = y[train_ID_index]


    test_X=X[test_ID_index]
#     test_Y = y.loc[test_ID_index].drop(columns=['subject_ID', 'walk_ID'],
#                inplace=False).values
    test_Y = y[test_ID_index]
    
#     train_X=preprocessing.normalize(train_X)
#     test_X=preprocessing.normalize(test_X)

    
#     rf = RandomForestRegressor(n_jobs=-1,n_estimators=400,max_depth=50,warm_start=True, random_state=1514)
    rf = xgb.XGBRegressor(alpha=0.1,reg_lambda=0.01)
    rf.fit(train_X,train_Y)
    pred_y=rf.predict(test_X)
    rmse = math.sqrt(mean_squared_error(pred_y, test_Y))
    res.append(rmse)



/home/gbalakrishnan/test_env/lib/python2.7/site-packages/ipykernel_launcher.py:20: FutureWarning: 'argmax' is deprecated, use 'idxmax' instead. The behavior of 'argmax'
will be corrected to return the positional maximum in the future.
Use 'series.values.argmax' to get the position of the maximum now.


Drop gyro_stdz_ct                   with p-value 0.999998
Drop gyro_signal_magnitude_area     with p-value 0.999993
Drop acc_energypeakZ_ct             with p-value 0.999994
Drop gyro_eclipse_c1                with p-value 0.999997
Drop acc_y_zero_crossing_rate       with p-value 0.999988
Drop gyro_rmsy_ct                   with p-value 0.999934
Drop gyro_stdx_ct                   with p-value 0.999994
Drop acc_eclipse_c1                 with p-value 0.999981
Drop gyro_sway_xz_percentile50      with p-value 0.999976
Drop gyro_eclipse_b1                with p-value 0.999958
Drop acc_b1_sway_per_second         with p-value 0.999997
Drop gyro_c1_sway_per_second        with p-value 0.999923
Drop gyro_rmsz_ct                   with p-value 0.999988
Drop gyro_sway_yz_percentile75      with p-value 0.999992
Drop gyro_energypeakX_ct            with p-value 0.999972
Drop gyro_energypeakY_ct            with p-value 0.999977
Drop gyro_b1_sway_per_second        with p-value 0.999832
Drop acc_eclip

Drop acc_eclipse_b1                 with p-value 0.999826
Drop gyro_x_energy                  with p-value 0.970763
Drop gyro_z_energy                  with p-value 0.992375
Drop acc_skz_ct                     with p-value 0.997384
Drop gyro_y_energy                  with p-value 0.98079
Drop gyro_kurty_ct                  with p-value 0.973931
Drop acc_z_zero_crossing_rate       with p-value 0.978489
Drop acc_kurtx_ct                   with p-value 0.992194
Drop acc_rmsz_ct                    with p-value 0.986732
Drop gyro_kurtx_ct                  with p-value 0.980856
Drop acc_sway_xz_percentile75       with p-value 0.997531
Drop gyro_sky_ct                    with p-value 0.989166
Drop acc_kurty_ct                   with p-value 0.993367
Drop gyro_z_zero_crossing_rate      with p-value 0.932438
Drop acc_y_energy                   with p-value 0.911585
Drop acc_rx_ct                      with p-value 0.972893
(2304, 14)
Drop gyro_stdz_ct                   with p-value 0.999998
Drop

Drop acc_sway_xz_percentile25       with p-value 0.999987
Drop gyro_stdy_ct                   with p-value 0.998257
Drop gyro_kurtz_ct                  with p-value 0.999675
Drop acc_sway_xz_percentile50       with p-value 0.999994
Drop gyro_ry_ct                     with p-value 0.999672
Drop acc_sway_yz_percentile25       with p-value 0.99997
Drop acc_rmsx_ct                    with p-value 0.999978
Drop acc_skx_ct                     with p-value 0.999572
Drop gyro_sway_xy_percentile75      with p-value 0.999539
Drop gyro_freqspeakX_ct             with p-value 0.995245
Drop gyro_skx_ct                    with p-value 0.998181
Drop acc_freqspeakX_ct              with p-value 0.891855
Drop acc_freqspeakY_ct              with p-value 0.975481
Drop gyro_freqspeakY_ct             with p-value 0.927454
Drop acc_rz_ct                      with p-value 0.938399
Drop acc_eclipse_b1                 with p-value 0.999826
Drop gyro_x_energy                  with p-value 0.970763
Drop gyro_z_ene

Drop gyro_eclipse_a1                with p-value 1.0
Drop acc_c1_sway_per_second         with p-value 0.999999
Drop acc_energypeakY_ct             with p-value 0.999997
Drop gyro_rz_ct                     with p-value 0.999999
Drop acc_stdy_ct                    with p-value 0.999997
Drop gyro_xz_sway_dispersion        with p-value 0.999964
Drop acc_stdz_ct                    with p-value 1.0
Drop acc_kurtz_ct                   with p-value 0.999994
Drop gyro_sway_xy_percentile25      with p-value 0.999997
Drop gyro_xy_sway_dispersion        with p-value 0.999932
Drop gyro_sway_xy_percentile50      with p-value 0.999937
Drop acc_a1_sway_per_second         with p-value 0.999973
Drop gyro_freqspeakZ_ct             with p-value 0.999946
Drop acc_sway_yz_percentile50       with p-value 0.999899
Drop acc_sky_ct                     with p-value 0.999991
Drop gyro_rx_ct                     with p-value 0.999592
Drop acc_sway_xz_percentile25       with p-value 0.999987
Drop gyro_stdy_ct       

Drop gyro_b1_sway_per_second        with p-value 0.999832
Drop acc_eclipse_a1                 with p-value 0.999971
Drop gyro_a1_sway_per_second        with p-value 0.999965
Drop gyro_sway_xz_percentile25      with p-value 0.999991
Drop gyro_rmsx_ct                   with p-value 0.999871
Drop acc_sway_xy_percentile50       with p-value 0.999993
Drop gyro_sway_xz_percentile75      with p-value 0.999781
Drop gyro_sway_yz_percentile25      with p-value 0.99995
Drop gyro_sway_yz_percentile50      with p-value 1.0
Drop gyro_energypeakZ_ct            with p-value 0.999955
Drop acc_energypeakX_ct             with p-value 0.999992
Drop gyro_yz_sway_dispersion        with p-value 0.999992
Drop gyro_skz_ct                    with p-value 0.999922
Drop acc_sway_xy_percentile25       with p-value 0.999997
Drop acc_rmsy_ct                    with p-value 1.0
Drop acc_signal_magnitude_area      with p-value 0.999992
Drop gyro_eclipse_a1                with p-value 1.0
Drop acc_c1_sway_per_second   

Drop gyro_stdz_ct                   with p-value 0.999998
Drop gyro_signal_magnitude_area     with p-value 0.999993
Drop acc_energypeakZ_ct             with p-value 0.999994
Drop gyro_eclipse_c1                with p-value 0.999997
Drop acc_y_zero_crossing_rate       with p-value 0.999988
Drop gyro_rmsy_ct                   with p-value 0.999934
Drop gyro_stdx_ct                   with p-value 0.999994
Drop acc_eclipse_c1                 with p-value 0.999981
Drop gyro_sway_xz_percentile50      with p-value 0.999976
Drop gyro_eclipse_b1                with p-value 0.999958
Drop acc_b1_sway_per_second         with p-value 0.999997
Drop gyro_c1_sway_per_second        with p-value 0.999923
Drop gyro_rmsz_ct                   with p-value 0.999988
Drop gyro_sway_yz_percentile75      with p-value 0.999992
Drop gyro_energypeakX_ct            with p-value 0.999972
Drop gyro_energypeakY_ct            with p-value 0.999977
Drop gyro_b1_sway_per_second        with p-value 0.999832
Drop acc_eclip

In [139]:
print(np.shape(X))

(2304, 14)


In [140]:
rf

XGBRegressor(alpha=0.1, base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
       reg_alpha=0, reg_lambda=0.01, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

In [141]:
res

[0.014430118430452303,
 0.013263504143319976,
 0.013269746054610464,
 0.011596134743183375,
 0.011401608780030532,
 0.011168742842025972,
 0.013952526852059737,
 0.014341998976471426,
 0.012652587683899118,
 0.010437497065430268]

In [44]:
split_rate=0.8

allIDs = np.unique(df['subject_ID'].values)
num_subjects = len(allIDs)
y = df[['subject_ID','walk_ID','BAC_VALUE']]
X = df.drop(columns=['subject_ID', 'walk_ID','BAC_VALUE' ],
           inplace=False)

In [127]:
np.mean(res)

0.0015804482708146072

In [92]:

id_string_selected = np.random.choice(allIDs, num_subjects, replace=False)
train_number = int(round(num_subjects*split_rate))
train_ID = np.random.choice(id_string_selected, train_number, replace=False)
test_ID = np.setdiff1d(id_string_selected,train_ID)
train_ID_index= df['subject_ID'].isin(train_ID)
test_ID_index = df['subject_ID'].isin(test_ID)

In [104]:
from sklearn import preprocessing

from sklearn.decomposition import PCA
X=preprocessing.normalize(X)
pca = PCA(2)
X = pca.fit_transform(X,y.values)

In [8]:
X

array([[6.09603215e-04, 3.25393683e-07, 7.72558952e-01, ...,
        3.85685598e-05, 1.38656296e-04, 7.85417548e-05],
       [5.52434192e-04, 9.47417994e-07, 7.32312774e-01, ...,
        3.74271710e-05, 1.25653009e-04, 7.11760531e-05],
       [5.68579720e-04, 1.17868506e-06, 7.58275685e-01, ...,
        4.09408186e-05, 1.29325364e-04, 7.32562556e-05],
       ...,
       [3.08754953e-03, 4.04715090e-05, 7.21738097e-01, ...,
        1.65813985e-04, 1.41149542e-03, 1.97958006e-04],
       [3.25353922e-03, 4.11082129e-05, 7.25808293e-01, ...,
        1.76519545e-04, 1.48737880e-03, 2.08600422e-04],
       [4.40954929e-03, 6.49619280e-05, 7.11909268e-01, ...,
        2.04170169e-04, 2.01585710e-03, 2.82717921e-04]])

In [70]:
train_ID

array(['ID_144', 'ID_179', 'ID_155', 'ID_112', 'ID_116', 'ID_140',
       'ID_157', 'ID_180', 'ID_145', 'ID_153', 'ID_137', 'ID_174',
       'ID_113', 'ID_128', 'ID_172', 'ID_148', 'ID_117', 'ID_159',
       'ID_141', 'ID_178', 'ID_149', 'ID_166', 'ID_118', 'ID_161',
       'ID_152', 'ID_132', 'ID_114', 'ID_168', 'ID_142', 'ID_136',
       'ID_129', 'ID_167', 'ID_135', 'ID_124', 'ID_162', 'ID_133',
       'ID_125', 'ID_138', 'ID_115', 'ID_151', 'ID_123', 'ID_177',
       'ID_127', 'ID_122', 'ID_165', 'ID_131', 'ID_146', 'ID_176',
       'ID_164'], dtype=object)

In [71]:
train_X=X.loc[train_ID_index].values
train_Y = y.loc[train_ID_index].drop(columns=['subject_ID', 'walk_ID'],
           inplace=False).values

test_X=X.loc[test_ID_index].values
test_Y = y.loc[test_ID_index].drop(columns=['subject_ID', 'walk_ID'],
           inplace=False).values
from sklearn import preprocessing
train_X=preprocessing.normalize(train_X)


In [72]:
import math
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.metrics import mean_squared_error, accuracy_score

rf = RandomForestRegressor(n_jobs=-1)
rf.fit(train_X,train_Y)
pred_y=rf.predict(test_X)
rmse = math.sqrt(mean_squared_error(pred_y, test_Y))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  


In [73]:
rmse

0.18207860780033605